<a href="https://colab.research.google.com/github/nunocesarsa/SENSECO_School_2021/blob/main/ColabNotebooks/SENSECO_04_RTM_Inversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setting up

- Now we just skim through all those intermediate steps without much explanation

## Setting up AutoSklearn

- Restart runtime after running cell

In [ ]:
#source: https://colab.research.google.com/github/vopani/fortyone/blob/main/notebooks/automl/tabular/Auto-Sklearn.ipynb#scrollTo=yFSo4OV0FOZN

!python3 -m pip install --upgrade pip
!pip3 install scikit-learn==0.24.1
!pip3 install pandas

!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip3 install
!pip3 install auto-sklearn

!pip install PipelineProfiler
#NOTICE: you might be required to restart runtime before proceeding


## RESTART RUNTIME!!

In [ ]:
#quick test:
from autosklearn.classification import AutoSklearnClassifier
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer(as_frame=True)['data']
target = load_breast_cancer(as_frame=True)['target']

sklearn_aml = AutoSklearnClassifier(time_left_for_this_task=30)
sklearn_aml.fit(X=data, y=target)

import PipelineProfiler
prof_data = PipelineProfiler.import_autosklearn(sklearn_aml)
PipelineProfiler.plot_pipeline_matrix(prof_data)

## Connecting to GDrive

In [3]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Setting up PROSAIL et al

In [ ]:
#Installing PROSAIL
!pip install prosail

#latin hypercube stuff
!pip install lhsmdu

#this package as a number of functions to deal with hyperspectral data
!pip install pysptools

## Importing packages

In [4]:
#General purpose: 
import matplotlib.pyplot as plt
import numpy
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats

#for data storage
import pickle 
import joblib

#the beutiful R like data frame
import pandas as pd

#PROSPECT+SAIL Radiative transfer mode package
import prosail

#Sampling design package
import lhsmdu

#a few more stuff for random
import random as rdm
import math

#package to for operations on spectral data
import pysptools as sptool 
from pysptools import distance

## AutoML
import autosklearn
from autosklearn.regression import AutoSklearnRegressor

## Scikit learn metrics stuff
import sklearn.metrics

## Loading *custom* PROSAIL Functions

- These are the same functions we used earlier

In [5]:
def custom_prosail(cab,lai,sol_zen,inc_zen,raa):
  
  import prosail
  #"default" parameters
  cm = 0.005
  cw = 0.005
  n= 1.
  car=10.
  cbrown=0.01
  typelidf=1 #this is the default option
  lidfa = -1 #leaf angle distribution parameter a and b
  lidfb=-0.15
  hspot= 0.01 #hotspot parameters 

  #sun and viewing angle
  #tts=30. #observation and solar position parameters
  #tto=10. 
  #psi=0.

  #for now i put them by hand but they should be an input of a custom function
  tts=sol_zen #solar zenith angle
  tto=inc_zen #sensor zenith angle
  psi=raa

  

  rho_out = prosail.run_prosail(n,
                                 cab,
                                 car,
                                 cbrown,
                                 cw,
                                 cm,
                                 lai,
                                 lidfa,
                                 hspot,
                                 tts,tto,psi,
                                 typelidf, lidfb,
                                 prospect_version="D",
                                 factor='SDR', 
                                 rsoil=.5, psoil=.5)
  return(rho_out)


#here you should change so it points to your file
filepath="/content/drive/MyDrive/SENSECO_S2Data/S2_Responses_S2A.csv"

def Prosail2S2(path2csv,spectra_input):

  #pointing needed packages
  import pandas as pd
  import numpy as np
  #upload a S2_Response.csv from https://earth.esa.int/web/sentinel/user-guides/sentinel-2-msi/document-library/-/asset_publisher/Wk0TKajiISaR/content/sentinel-2a-spectral-responses

  s2_table = pd.read_csv(path2csv,sep=";",decimal=",") #check if this is proper, regarding the sep and dec
  #chekc which row you are actually extracting

  s2_table_sel = s2_table[s2_table['SR_WL'].between(400,2500)] #selects all values between 400 and 2500
  spectra_input_df = pd.DataFrame(data=spectra_input,columns=["rho"],index=s2_table_sel.index) #transforms the input array into a pandas df with the column name rho and row.index = to the original input table

  rho_s2 = s2_table_sel.multiply(spectra_input_df['rho'],axis="index") #calculates the numerator
  w_band_sum = s2_table_sel.sum(axis=0,skipna = True) #calculates the denominator

  output = (rho_s2.sum(axis=0)/w_band_sum).rename_axis("ID").values #runs the weighted mean and converts the output to a numpy array

  return output[1:] #removes the first value because it represents the wavelength column


def Gen_spectra_data(input_param_table,doPlot=False,bandlist=[0,1,2,3,4,5,6,7,8,9,10,11,12]):
  k = 1
  #pd_train_traits=traits
  #print(range(len(traits)))
  import matplotlib.pyplot as plt
  import pandas as pd
  import numpy as np
  
  for i in range(len(input_param_table)):

    #edit this section accordingly
    filepath = "/content/drive/MyDrive/SENSECO_S2Data/S2_Responses_S2A.csv"

    #vegetation parametrs
    cab_t = input_param_table["cab"][i]
    lai_t = input_param_table["lai"][i]

    #observation parameters
    sol_zen_t = input_param_table["sol_zen"][i]
    inc_zen_t = input_param_table["inc_zen"][i]
    raa_t     = input_param_table["raa"][i]

    if k == 1:

      tr_rho_s = custom_prosail(cab_t,lai_t,sol_zen_t,inc_zen_t,raa_t)
      tr_rho_s = Prosail2S2(filepath,tr_rho_s)[bandlist,]

      if doPlot == True:
        x = bandlist
        plt.plot ( x, tr_rho_s)
        #plt.legend(loc='best')
      
    if k > 1:
      tr_rho_t = custom_prosail(cab_t,lai_t,sol_zen_t,inc_zen_t,raa_t)
      tr_rho_t = Prosail2S2(filepath,tr_rho_t)[bandlist,]
      tr_rho_s = np.vstack((tr_rho_s,tr_rho_t))

      if doPlot == True:
        plt.plot ( x, tr_rho_t)

    k = k+1

  rho_samples=tr_rho_s
  return rho_samples

def combined_noise(ref,sigma):

  #ref is an input vector and sigma is the standar deviation of the noise - there is a possibilityu of values going over over 1 or lower than 0 But those values are corrected in the end. 
  
  mp_noise = ref*(1 + stats.truncnorm.rvs(0-sigma*4,0+sigma*4,loc=0,scale=sigma*2,size=ref.shape[0]))
  ad_noise = 0+stats.truncnorm.rvs(0-sigma*2,0+sigma*2,loc=0,scale=sigma,size=ref.shape[0])
  out_ref = mp_noise+ ad_noise

  #making everything that goes under, be 0 or 1
  out_ref[out_ref>1]=1
  out_ref[out_ref<0]=0

  return out_ref

### Quick test:

- just to make sure everything is ready to go

In [ ]:
#generating LHS random matrix
LHS_train = lhsmdu.createRandomStandardUniformMatrix(5,10)

#then we convert this into a Pandas table and rename it
pd_param_input = pd.DataFrame.transpose(pd.DataFrame(LHS_train))
pd_param_input.columns = ["cab","lai","sol_zen","inc_zen","raa"]

#vegetation parameters
pd_param_input["cab"]    =pd_param_input["cab"]*80+20
pd_param_input["lai"]    =pd_param_input["lai"]*6+1

#observation parameters - for now very small and basic
pd_param_input["sol_zen"]=pd_param_input["sol_zen"]*31+29
pd_param_input["inc_zen"]=pd_param_input["inc_zen"]*31+29
pd_param_input["raa"]    =pd_param_input["raa"]*31+29

#test the function
test_spectra = Gen_spectra_data(pd_param_input,doPlot=True,bandlist=[1,2,3,4,5,6,7,11,12])

### Adding noise:

In [ ]:
#adding noise:
test_spectra_noise = np.apply_along_axis(combined_noise,1,test_spectra,sigma=0.05)
test_spectra_noise

# Generating data

- Now, we want to generate a N number of samples that will be used for training the actual models for RTM inversion

In [9]:
n_samples = 1000

#generating LHS random matrix
LHS_train = lhsmdu.createRandomStandardUniformMatrix(5,n_samples)

#then we convert this into a Pandas table and rename it
pd_param_input = pd.DataFrame.transpose(pd.DataFrame(LHS_train))
pd_param_input.columns = ["cab","lai","sol_zen","inc_zen","raa"]

#vegetation parameters
pd_param_input["cab"]    =pd_param_input["cab"]*60+20
pd_param_input["lai"]    =pd_param_input["lai"]*6+1

#observation parameters - Based on the data we will use later
pd_param_input["sol_zen"]=pd_param_input["sol_zen"]*45+15 # the average is aroudn 39 but the maximum is just below 60 and the minimum is 18.
pd_param_input["inc_zen"]=pd_param_input["inc_zen"]*2+3 # the average is around 4 - so now the data varies between 3 and 5
pd_param_input["raa"]    =pd_param_input["raa"]*50+80 # the avreage is aroudn 100 but the maximum is 128 and the min 84. 

#generating 1000 samples
np_spectra = Gen_spectra_data(pd_param_input,bandlist=[1,2,3,4,5,6,7,11,12])

#adding noise - comment this line if you want to try a pure spectra inversion
np_spectra = np.apply_along_axis(combined_noise,1,np_spectra,sigma=0.05)

#creating a pandas table because why not 
pd_spectra = pd.DataFrame(data=np_spectra,columns=["B02","B03","B04",
                                                   "B05","B06","B07",
                                                   "B8A","B11","B12"])

#combines all the data into one single csv
pd_alldata = pd.concat([pd_param_input, pd_spectra], axis=1)

#first step is always dividing the dataset between validation and training:
pd_train = pd_alldata.sample(frac=0.8,random_state=200)
pd_test  = pd_alldata.drop(pd_train.index)
 

In [ ]:
pd_alldata.describe()

#AutoSklearn

FYI: https://automl.github.io/auto-sklearn/master/_modules/autosklearn/estimators.html

## Data set up

In [12]:
#Autosklearn expects input data as: x, y in separate datasets
pd_train_y = pd_train[["cab","lai"]]
pd_train_x = pd_train[["sol_zen","inc_zen","raa","B02","B03","B04","B05","B06","B07","B8A","B11","B12"]]

pd_test_y = pd_test[["cab","lai"]]
pd_test_x = pd_test[["sol_zen","inc_zen","raa","B02","B03","B04","B05","B06","B07","B8A","B11","B12"]]


# First model:

In [ ]:
#Default settings (except for time)
AutoRTM_0030 = autosklearn.regression.AutoSklearnRegressor(time_left_for_this_task=30)
AutoRTM_0030.fit(pd_train_x,pd_train_y,pd_test_x,pd_test_y)

## Checking error

In [ ]:
predictions_0030 = AutoRTM_0030.predict(pd_test_x)
print("Cab MAE:", sklearn.metrics.mean_absolute_error(pd_test_y["cab"], predictions_0030[:,0]))
print("LAI MAE:", sklearn.metrics.mean_absolute_error(pd_test_y["lai"], predictions_0030[:,1]))

## Exploring pipeline

In [ ]:
AutoRTM_0030_pipeline = PipelineProfiler.import_autosklearn(AutoRTM_0030)
PipelineProfiler.plot_pipeline_matrix(AutoRTM_0030_pipeline)

## Plotting

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2)
ax1.plot(predictions_0030[:,0],pd_test_y["cab"],"o")
ax2.plot(predictions_0030[:,1],pd_test_y["lai"],"o")



# Second model

-  here we increase the time for 5 minutes

In [ ]:
#Default settings (except for time)
AutoRTM_0300 = autosklearn.regression.AutoSklearnRegressor(time_left_for_this_task=300)
AutoRTM_0300.fit(pd_train_x,pd_train_y,pd_test_x,pd_test_y)

predictions_0300 = AutoRTM_0300.predict(pd_test_x)
print("Cab MAE:", sklearn.metrics.mean_absolute_error(pd_test_y["cab"], predictions_0300[:,0]))
print("LAI MAE:", sklearn.metrics.mean_absolute_error(pd_test_y["lai"], predictions_0300[:,1]))

In [ ]:
AutoRTM_0300_pipeline = PipelineProfiler.import_autosklearn(AutoRTM_0300)
PipelineProfiler.plot_pipeline_matrix(AutoRTM_0300_pipeline)

Gaussian processes @ autosklearn github:

 https://github.com/automl/auto-sklearn/blob/master/autosklearn/pipeline/components/regression/gaussian_process.py

The model optimizes the lenghscale bounds: https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.kernels.RBF.html#sklearn.gaussian_process.kernels.RBF





In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2)
ax1.plot(predictions_0300[:,0],pd_test_y["cab"],"o")
ax2.plot(predictions_0300[:,1],pd_test_y["lai"],"o")

### Accuracy over time

Using the suggestion from the manual: https://automl.github.io/auto-sklearn/master/examples/40_advanced/example_pandas_train_test.html

In [21]:
from smac.tae import StatusType
import time

def get_runhistory_models_performance(automl):
    metric = AutoRTM_0300.automl_._metric
    data = automl.automl_.runhistory_.data
    performance_list = []
    for run_key, run_value in data.items():
        if run_value.status != StatusType.SUCCESS:
            # Ignore crashed runs
            continue
        # Alternatively, it is possible to also obtain the start time with ``run_value.starttime``
        endtime = pd.Timestamp(time.strftime('%Y-%m-%d %H:%M:%S',
                                             time.localtime(run_value.endtime)))
        val_score = metric._optimum - (metric._sign * run_value.cost)
        test_score = metric._optimum - (metric._sign * run_value.additional_info['test_loss'])
        train_score = metric._optimum - (metric._sign * run_value.additional_info['train_loss'])
        performance_list.append({
            'Timestamp': endtime,
            'single_best_optimization_score': val_score,
            'single_best_test_score': test_score,
            'single_best_train_score': train_score,
        })
    return pd.DataFrame(performance_list)

In [ ]:
ensemble_performance_frame = pd.DataFrame(AutoRTM_0300.automl_.ensemble_performance_history)
best_values = pd.Series({'ensemble_optimization_score': -np.inf,
                         'ensemble_test_score': -np.inf})

for idx in ensemble_performance_frame.index:
    if (
        ensemble_performance_frame.loc[idx, 'ensemble_optimization_score']
        > best_values['ensemble_optimization_score']
    ):
        best_values = ensemble_performance_frame.loc[idx]
    ensemble_performance_frame.loc[idx] = best_values

individual_performance_frame = get_runhistory_models_performance(AutoRTM_0300)
best_values = pd.Series({'single_best_optimization_score': -np.inf,
                         'single_best_test_score': -np.inf,
                         'single_best_train_score': -np.inf})
for idx in individual_performance_frame.index:
    if (
        individual_performance_frame.loc[idx, 'single_best_optimization_score']
        > best_values['single_best_optimization_score']
    ):
        best_values = individual_performance_frame.loc[idx]
    individual_performance_frame.loc[idx] = best_values

pd.merge(
    ensemble_performance_frame,
    individual_performance_frame,
    
    on="Timestamp", how='outer'
).sort_values('Timestamp').fillna(method='ffill').plot(
    x='Timestamp',
    kind='line',
    legend=True,
    title='Auto-sklearn accuracy over time',
    grid=True,
    figsize=(10,10),
)

plt.show()

#!!!Problem!!!

**Gaussian process for RTM inversion can be prone to overfitting**. Often it works well with "pure" inversion but can easily produce "wacky" results when using real data.

This can be avoided by adding some "noise" to the data and by adding some "leeway" to the alpha parameter.

https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html#sklearn.gaussian_process.GaussianProcessRegressor

Alpha: "*Value added to the diagonal of the kernel matrix during fitting. This can prevent a potential numerical issue during fitting, by ensuring that the calculated values form a positive definite matrix. It can also be interpreted as the variance of additional Gaussian measurement noise on the training observations.*"


To solve that, we need to dwelve a bit deeper into autosklearn and create a new model classs that will become available for use. 

##Advanced - Adding new estimators:

Changing AutoSklearn so we can configure a bit the hyperparameter space

Adapting from:
https://automl.github.io/auto-sklearn/master/examples/80_extending/example_restrict_number_of_hyperparameters.html#sphx-glr-examples-80-extending-example-restrict-number-of-hyperparameters-py

List of Default regression models: https://github.com/automl/auto-sklearn/tree/master/autosklearn/pipeline/components/regression

## New GP class: 

- original class: https://github.com/automl/auto-sklearn/blob/master/autosklearn/pipeline/components/regression/gaussian_process.py

- Useful resource: https://github.com/automl/auto-sklearn/blob/master/examples/80_extending/example_extending_regression.py

- objective is to force a minimum 0.05 alpha parameter so we dont risk overfitting the artificial data

In [23]:
from ConfigSpace.configuration_space import ConfigurationSpace
from ConfigSpace.hyperparameters import UniformFloatHyperparameter

from autosklearn.pipeline.components.base import AutoSklearnRegressionAlgorithm
from autosklearn.pipeline.constants import DENSE, UNSIGNED_DATA, PREDICTIONS

class GaussianProcess2(AutoSklearnRegressionAlgorithm):
    def __init__(self, alpha, thetaL, thetaU, random_state=None):
        self.alpha = alpha
        self.thetaL = thetaL
        self.thetaU = thetaU
        # We ignore it
        self.random_state = random_state
        self.estimator = None
        self.scaler = None

    def fit(self, X, y):
        import sklearn.gaussian_process

        self.alpha = float(self.alpha)
        self.thetaL = float(self.thetaL)
        self.thetaU = float(self.thetaU)

        n_features = X.shape[1] 
        kernel = sklearn.gaussian_process.kernels.RBF( #here we could change the kernel parameters - notice also that deeper changes are needed to allow for kernel search
            length_scale=[1.0]*n_features,
            length_scale_bounds=[(self.thetaL, self.thetaU)]*n_features)

        # Instanciate a Gaussian Process model
        self.estimator = sklearn.gaussian_process.GaussianProcessRegressor(
            kernel=kernel,
            n_restarts_optimizer=10,
            optimizer='fmin_l_bfgs_b',
            alpha=self.alpha,
            copy_X_train=True,
            random_state=self.random_state,
            normalize_y=True)

        self.estimator.fit(X, y)
        return self

    def predict(self, X):
        if self.estimator is None:
            raise NotImplementedError
        return self.estimator.predict(X)

    @staticmethod
    def get_properties(dataset_properties=None):
        return {'shortname': 'GP',
                'name': 'Gaussian Process',
                'handles_regression': True,
                'handles_classification': False,
                'handles_multiclass': False,
                'handles_multilabel': False,
                'handles_multioutput': True,
                'is_deterministic': True,
                'input': (DENSE, UNSIGNED_DATA),
                'output': (PREDICTIONS,)}

    @staticmethod
    def get_hyperparameter_search_space(dataset_properties=None):
        alpha = UniformFloatHyperparameter(
            name="alpha", lower=0.05, upper=1.0, default_value=0.05, log=True) #NOTICE HERE - i am forcing a lower bound of 0.05 for the alpha parameter
        thetaL = UniformFloatHyperparameter(
            name="thetaL", lower=1e-10, upper=1e-3, default_value=1e-6, log=True)
        thetaU = UniformFloatHyperparameter(
            name="thetaU", lower=1.0, upper=100000, default_value=100000.0, log=True)

        cs = ConfigurationSpace()
        cs.add_hyperparameters([alpha, thetaL, thetaU])
        return cs

#Pushing the changes

In [ ]:
autosklearn.pipeline.components.regression.add_regressor(GaussianProcess2)
cs = GaussianProcess2.get_hyperparameter_search_space()
print(cs)

## Restricted model:

In [ ]:

#we can FORCE autosklearn to try ONLY specific models. And we will use this to compare RF against GP
AutoRTM_Restricted = autosklearn.regression.AutoSklearnRegressor(time_left_for_this_task=120,
                                                                 include_estimators=["GaussianProcess2", "gaussian_process","random_forest" ])
AutoRTM_Restricted.fit(pd_train_x,pd_train_y,pd_test_x,pd_test_y)

predictions_rest = AutoRTM_Restricted.predict(pd_test_x)
print("Cab MAE:", sklearn.metrics.mean_absolute_error(pd_test_y["cab"], predictions_rest[:,0]))
print("LAI MAE:", sklearn.metrics.mean_absolute_error(pd_test_y["lai"], predictions_rest[:,1]))

In [ ]:
AutoRTM_Restricted_pipeline = PipelineProfiler.import_autosklearn(AutoRTM_Restricted)
PipelineProfiler.plot_pipeline_matrix(AutoRTM_Restricted_pipeline)

## Plotting example:

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2)
ax1.plot(predictions_rest[:,0],pd_test_y["cab"],"o")
ax2.plot(predictions_rest[:,1],pd_test_y["lai"],"o")

#Saving models:

In [28]:
import joblib
joblib.dump(AutoRTM_0030, '/content/drive/MyDrive/SENSECO/Models/AutoRTM_0030.pkl')
joblib.dump(AutoRTM_0300, '/content/drive/MyDrive/SENSECO/Models/AutoRTM_0300.pkl')
joblib.dump(AutoRTM_Restricted, '/content/drive/MyDrive/SENSECO/Models/AutoRTM_Restricted.pkl')

['/content/drive/MyDrive/SENSECO/Models/AutoRTM_Restricted.pkl']

In [29]:
#Loading example and confirming it loaded the right model

loaded_mdl =  joblib.load('/content/drive/MyDrive/SENSECO/Models/AutoRTM_Restricted.pkl')

In [ ]:
PipelineProfiler.plot_pipeline_matrix(AutoRTM_Restricted_pipeline)

In [ ]:
loaded_mdl_pipeline = PipelineProfiler.import_autosklearn(loaded_mdl)
PipelineProfiler.plot_pipeline_matrix(loaded_mdl_pipeline)